In [ ]:
import czifile
import numpy as np

import skimage.io
import skimage.filters
import skimage.feature
import skimage.morphology

import scipy

import bebi103

import holoviews as hv
hv.extension('bokeh')
import colorcet

import bokeh.io
bokeh.io.output_notebook()

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.13, 0.13],
    cmap=None,
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=200,
        title=titles[0],
        cmap=cmap,
        interpixel_distance=interpixel_distances[0],
        length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=200,
        title=titles[1],
        cmap=cmap,
        interpixel_distance=interpixel_distances[1],
        length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def dapi_sort(image):
    return image[0,0,0,0,:,:,:,0]

def image_reducer(filepath, zoom):
    img = czifile.imread(filepath)
    img = dapi_sort(img)
    img = np.sum(img, 0)
    
    return img[zoom]
        
zoom1 = np.s_[1500:3000, :1500]

In [ ]:
im1 = image_reducer('20.02.15_DAPI-seg-1.czi', zoom1)

im1.shape

In [ ]:
hv.Image(im1
).opts(
    height = 500,
    width = 500,
    cmap = 'viridis')

In [ ]:
# Convert image to float
im_float = (im1.astype(float) - im1.min()) / (im1.max() - im1.min())

In [ ]:
# Compute LoG
im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, 20)

# Check out results
bokeh.io.show(
    bebi103.image.imshow(
        im_LoG,
        cmap=colorcet.coolwarm,
        interpixel_distance=0.65,
        length_units="µm",
        colorbar=True,
    )
)

In [ ]:
selem = skimage.morphology.square(20)

# Do max filter and min filter
im_LoG_max = scipy.ndimage.filters.maximum_filter(im_LoG, footprint=selem)
im_LoG_min = scipy.ndimage.filters.minimum_filter(im_LoG, footprint=selem)

# Image of zero-crossings
im_edge = ((im_LoG >= 0) & (im_LoG_min < 0)) | ((im_LoG <= 0) & (im_LoG_max > 0))

# Show result
bokeh.io.show(show_two_ims(im_float, im_edge, titles=["original", "edges"]))

In [ ]:
def zero_crossing_filter(im, thresh):
    """
    Returns image with 1 if there is a zero crossing and 0 otherwise.

    thresh is the the minimal value of the gradient, as computed by Sobel
    filter, at crossing to count as a crossing.
    """
    # Square structuring element
    selem = skimage.morphology.square(3)

    # Do max filter and min filter
    im_max = scipy.ndimage.filters.maximum_filter(im, footprint=selem)
    im_min = scipy.ndimage.filters.minimum_filter(im, footprint=selem)

    # Compute gradients using Sobel filter
    im_grad = skimage.filters.sobel(im)

    # Return edges
    return ( (  ((im >= 0) & (im_min < 0))
              | ((im <= 0) & (im_max > 0)))
            & (im_grad >= thresh) )

In [ ]:
# Find zero-crossings
im_edge = zero_crossing_filter(im_LoG, 0.000001)

# Show result
bokeh.io.show(show_two_ims(im_float, im_edge, titles=["original", "edges"]))

In [ ]:
# Fill holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Remove small objectes that are not bacteria
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=1000)

# Show result
bokeh.io.show(
    show_two_ims(im_float, im_bw, titles=["original", "segmented"])
)

In [ ]:
im_labeled, n_labels = skimage.measure.label(im_bw, background=0, return_num=True)

# Show number of nuclei
print("Number of individual nuclei = ", n_labels)

# See result (one of the few times it's ok to use rainbow colormap!)
bokeh.io.show(
    bebi103.image.imshow(
        im_labeled, cmap=colorcet.b_glasbey_hv, interpixel_distance=0.65, length_units="µm"
    )
)